---
Коллеги из ML-отдела планируют выкатывать новый алгоритм, рекомендующий нашим пользователям интересные посты. 

- Алгоритм добавляет пользователям 1-2 просмотра
- Вероятность того, что он сработает, составляет 90%
- Если у пользователя меньше 50 просмотров, то алгоритм не сработает

  
Предполагаю гипотезу, что увеличение числа просмотров приведёт и к увеличению лайков на пользователя. 
Проверю будут ли различия в среднем количестве лайков на пользователя, для этого проведу симуляцию *Монте-Карло*

---

In [33]:
import pandas as pd
import pandahouse as ph
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, ttest_ind

#подключение к бд
from dotenv import load_dotenv
import os
import clickhouse_connect

In [34]:
connection = clickhouse_connect.get_client(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    username=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_DATABASE")
)

In [35]:
q="""
SELECT 
    user_id, 
    sum(action = 'like') as likes,
    sum(action = 'view') as views
FROM simulator_20251220.feed_actions
WHERE toDate(time) BETWEEN '2025-11-14' AND '2025-11-20'
GROUP BY user_id
"""

In [36]:
df=ph.read_clickhouse(q, connection= connection)
df

,user_id,likes,views
0,13289,2,32
1,121096,12,57
2,5090,2,14
3,129283,23,80
4,4394,25,54
...,...,...,...
41992,3795,3,17
41993,123954,23,101
41994,124666,32,141
41995,11280,29,133


In [37]:
#На эксперимент нам выделили неделю. 
#Допустим, что за эту неделю в наш сервис зайдёт столько же пользователей, сколько зашло в период АА-теста. 
#Мы планируем разбивать пользователей на две группы в соотношении 50/50. 
#Посчитаем, сколько пользователей в таком случае придётся на одну группу.
group_size=df.user_id.nunique()//2
group_size

20998

In [38]:
q1="""
SELECT
    views,
    count() AS users
FROM (
    SELECT
        user_id,
        sum(action = 'like') AS likes,
        sum(action = 'view') AS views
    FROM simulator_20251220.feed_actions
    WHERE toDate(time) BETWEEN '2025-11-14' AND '2025-11-20'
    GROUP BY user_id
)
GROUP BY views
ORDER BY views
"""

In [39]:
#создаём генератор псевдослучайных чисел
rng = np.random.default_rng()

In [40]:
views_distribution=ph.read_clickhouse(q1, connection= connection)
views_distribution

,views,users
0,1,4
1,2,1
2,3,4
3,4,5
4,5,18
...,...,...
296,324,1
297,326,1
298,355,1
299,364,1


In [41]:
views_distribution['p'] = views_distribution['users']/views_distribution.users.sum()

In [42]:
q2="""
SELECT 
    floor(ctr, 2) as ctr, count() as users
FROM (
    SELECT toDate(time) as dt,
        user_id, 
        sum(action = 'like') / sum(action = 'view') as ctr
    FROM simulator_20251220.feed_actions
    WHERE toDate(time) BETWEEN '2025-11-14' AND '2025-11-20'
    GROUP BY user_id, dt
)
group by ctr
"""


In [43]:
ctr_distribution=ph.read_clickhouse(q2, connection= connection)
ctr_distribution['p'] = ctr_distribution['users']/ ctr_distribution['users'].sum()

In [45]:
from tqdm import tqdm

n_simulations = 20000
success_count = 0

# Запускаем цикл симуляций
for _ in tqdm(range(n_simulations)):
    
    # 1. Сэмплируем базовые параметры для Группы А и Б из распределений
    group_A_views = rng.choice(views_distribution['views'], size=group_size, p=views_distribution['p'], replace=True)
    group_B_views = rng.choice(views_distribution['views'], size=group_size, p=views_distribution['p'], replace=True)
    group_A_ctr = rng.choice(ctr_distribution['ctr'], size=group_size, p=ctr_distribution['p'], replace=True)
    group_B_ctr = rng.choice(ctr_distribution['ctr'], size=group_size, p=ctr_distribution['p'], replace=True)
    
    # 2. Применяем эффект алгоритма в Группе Б (согласно условию)
    # n=1 в binomial — это способ подбросить монетку для каждого пользователя
   
    extra_views = group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=group_size)) * rng.binomial(n=1, p=0.9, size=group_size) * (group_B_views >= 50))
    group_A_likes = rng.binomial(n=group_A_views.astype(int), p=group_A_ctr)
    group_B_likes = rng.binomial(n=extra_views.astype(int), p=group_B_ctr)
    
    p_val = stats.ttest_ind(group_A_likes, group_B_likes, equal_var=False).pvalue
    if p_val < 0.05:
        success_count += 1

power = (success_count / n_simulations) * 100
print(f"Мощность: {round(power, 1)}%")
    

100%|██████████| 20000/20000 [05:41<00:00, 58.59it/s]

Мощность: 26.2%


---
Получили очень низкий уровень мощности.

Это значит, что при таком размере выборки и эффекте алгоритма статистически значимое увеличение лайков сможем обнаружить только в **26.2%** случаев (только у четверти пользователей).

Основная причина низкой мощности - малый эффект алгоритма (только 1–2 просмотра на фоне пользователей с уже большим количеством просмотров) и высокая дисперсия лайков на пользователя.

Возможное решение: 
- распространить эффект алгоритма на пользователей с меньшим количеством лайков (50 -> 35-30)
- увеличить длительность эксперимента

---